<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [92]:
import requests
import pandas as pd

In [93]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [128]:
data[2]

'From fairest creatures we desire increase,'

In [129]:
len(data)

169308

In [98]:
#Shakespeare Data Parsed by Play
df_toc.head(20)

,title,start,end,text
0,THE TRAGEDY OF ANTONY AND CLEOPATRA,-99,14379,
1,AS YOU LIKE IT,14380,17171,AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2,THE COMEDY OF ERRORS,17172,20372,THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3,THE TRAGEDY OF CORIOLANUS,20373,30346,THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4,CYMBELINE,30347,30364,CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...
5,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK",30365,37051,"THE TRAGEDY OF HAMLET, PRINCE OF DENMARK\r\n\r..."
6,THE FIRST PART OF KING HENRY THE FOURTH,37052,41767,THE FIRST PART OF KING HENRY THE FOURTH\r\n\r\...
7,THE SECOND PART OF KING HENRY THE FOURTH,41768,-100,THE SECOND PART OF KING HENRY THE FOURTH\r\n\r...
8,THE LIFE OF KING HENRY THE FIFTH,-99,45176,
9,THE LIFE OF KING HENRY V,45177,53383,THE LIFE OF KING HENRY V\r\n\r\n\r\n\r\nConten...


In [95]:
#loc to see what text looks like
df_toc['text'][1][:100]

'AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r\n\r\n  DUKE, living in exile\r\n  FREDERICK, his brother, and usu'

In [96]:
len(data)

169308

In [97]:
data[3]

'That thereby beauty’s rose might never die,'

### Generate Text

In [101]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping, TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import random
import sys
import os
import datetime

In [100]:
# Find Hamlet
df_toc[df_toc['title'].str.match('THE TRAGEDY OF JULIUS CAESAR')]

,title,start,end,text
14,THE TRAGEDY OF JULIUS CAESAR,66784,71427,THE TRAGEDY OF JULIUS CAESAR\r\n\r\n\r\n\r\nCo...


In [102]:
locations[14]

{'title': 'THE TRAGEDY OF JULIUS CAESAR', 'start': 66784}

In [104]:
TRAGEDY = df_toc['text'][14]
len(TRAGEDY)

121284

In [106]:
chars = list(set(TRAGEDY))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

Our corpus contains 72 unique characters.


In [107]:
# Create the Sequence Data

maxlen = 150
step = 1

encoded = [char_int[c] for c in TRAGEDY]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))

sequences: 121134


In [108]:
# Specify x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [109]:
x.shape, y.shape

((121134, 150, 72), (121134, 72))

In [110]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars)), dropout=0.2))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [111]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 256)               336896    
_________________________________________________________________
dense_3 (Dense)              (None, 72)                18504     
Total params: 355,400
Trainable params: 355,400
Non-trainable params: 0
_________________________________________________________________


In [117]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [118]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(TRAGEDY) - maxlen - 1)
    
    generated = ''
    
    sentence = TRAGEDY[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: ----- \n')
    sys.stdout.write(generated)
    
    print('\n\n\n-----New text: -----')

    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
    print()
    sys.stdout.flush()
    print('\n\n')
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [121]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

model.fit(x, y,
          batch_size=32,
          validation_split=.2,
          epochs=10,
          callbacks=[print_callback, 
                     #EarlyStopping(min_delta=.02, monitor='val_loss', patience=10),
                     tensorboard_callback])

Epoch 1/10
3029/3029 [==============================] - ETA: 0s - loss: 2.3504
----- Generating text after Epoch: 0
----- Generating with seed: ----- 

’d.

ARTEMIDORUS.
Delay not, Caesar. Read it instantly.

CAESAR.
What, is the fellow mad?

PUBLIUS.
Sirrah, give place.

CASSIUS.
What, ur


-----New text: -----
enysu s oond supsure.

F[_tureteas. I
Hellyttes Than’deUtiBr ene ait yst And no honm

inRSinkerlltsh anChese lestme he yould
As Irouto the bush’l the Cassar.
Eobeyou mellt, [ ous on wirl on Loubou mom nom enaesprt
Asd sete’s asfon ne you salh this so?h

BRUTUS.
Deloursnowtert on,
Cabear sole ere sech ut you the hirelos refshat amd CBoCpA.
Imchas tter tun shi hallut ansthe?
O whitear. 



3029/3029 [==============================] - 917s 303ms/step - loss: 2.3504 - val_loss: 2.0237
Epoch 2/10
3029/3029 [==============================] - ETA: 0s - loss: 2.0382
----- Generating text after Epoch: 1
----- Generating with seed: ----- 

ed and valiant soldier.

ANTONY.
So is my horse,

In [122]:
%load_ext tensorboard

In [123]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 5188), started 3:54:23 ago. (Use '!kill 5188' to kill it.)

In [134]:
text = '\r\n'.join(text)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

print(f"Our corpus contains {len(chars)} unique characters.")

NameError: name 'text' is not defined

In [ ]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

In [ ]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    y[i, next_char[i]] = 1

In [ ]:
x.shape

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='adam'))
model.compile(loss='categorical_crossentropy', optimizer='Nadam')

In [ ]:
model.summary()

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
model.fit(x, y,
          batch_size=128,
          epochs=50,
          callbacks=[print_callback],
          )

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN